# Simple Example

Let's load our quickstart and translate to spanish

In [1]:
from pathlib import Path

In [2]:
dict_file = Path("configs/language_dicts/es.yaml")
human_file = Path("configs/human_prompt.txt")
system_file = Path("configs/system_prompt.txt")
md_file = Path("docs/quickstart.md")

In [3]:
with open(dict_file, "r") as f:
    es_dict = f.read()

with open(md_file, "r") as f:
    quickstart = f.read()

In [4]:
from gpt_translate.prompts import PromptTemplate
from gpt_translate.loader import remove_markdown_comments, split_markdown
from gpt_translate.utils import count_tokens

In [5]:
p = PromptTemplate.from_files(system_file, human_file, dict_file)

Let's remove comments to save tokens

In [7]:
cleaned_quickstart = remove_markdown_comments(quickstart)

spliting into chunks!

In [8]:
splitted_markdown = split_markdown(cleaned_quickstart)

In [9]:
splitted_markdown

["---\ndescription: W&B Quickstart.\ndisplayed_sidebar: default\n---\n\nimport Tabs from '@theme/Tabs';\nimport TabItem from '@theme/TabItem';",
 '# Quickstart\n\nInstall W&B and start tracking your machine learning experiments in minutes.',
 '## 1. Create an account and install W&B\nBefore you get started, make sure you create an account and install W&B:\n\n1. [Sign up](https://wandb.ai/site) for a free account at [https://wandb.ai/site](https://wandb.ai/site) and then login to your wandb account.  \n2. Install the wandb library on your machine in a Python 3 environment using [`pip`](https://pypi.org/project/wandb/).  \n\n\nThe following code snippets demonstrate how to install and log into W&B using the W&B CLI and Python Library:\n\n<Tabs\n  defaultValue="notebook"\n  values={[\n    {label: \'Notebook\', value: \'notebook\'},\n    {label: \'Command Line\', value: \'cli\'},\n  ]}>\n  <TabItem value="cli">\n\nInstall the CLI and Python library for interacting with the Weights and Bias

In [10]:
count_tokens(splitted_markdown[0])

33

In [11]:
from openai import OpenAI
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)

client = OpenAI()

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

In [12]:
def concat_md_chunks(chunks, sep="\n\n"):
    return sep.join(chunks)

In [16]:
def translate_chunk(chunk, prompt):
    res = completion_with_backoff(
        model="gpt-4", 
        messages=prompt.format(md_chunk=chunk))

    return res.choices[0].message.content

In [17]:
def translate_splitted_md(splitted_markdown, prompt, max_chunk_tokens=400, sep="\n\n"):
    "Translate a list of markdown chunks"
    packed_chunks = ""
    translated_file = ""
    packed_chunks_len = 0

    for i, chunk in enumerate(splitted_markdown):
        
        n_tokens = count_tokens(chunk)

        if packed_chunks_len + n_tokens <= max_chunk_tokens:
            print(f"Packing chunk {i} with {n_tokens} tokens")
            packed_chunks += sep + chunk
            packed_chunks_len += n_tokens
        else:
            print(f">> Translating {packed_chunks_len} tokens")
            t_chunk = translate_chunk(packed_chunks, p)
            translated_file += sep + t_chunk
            print(f"Packing chunk {i} with {n_tokens} tokens")
            packed_chunks = chunk
            packed_chunks_len = n_tokens

    return translated_file

In [18]:
spanish_md = translate_splitted_md(splitted_markdown[0:4], p, max_chunk_tokens=128)

Packing chunk 0 with 33 tokens
Packing chunk 1 with 17 tokens
>> Translating 50 tokens
Packing chunk 2 with 251 tokens
>> Translating 251 tokens
Packing chunk 3 with 190 tokens


In [19]:
print(concat_md_chunks(splitted_markdown[0:4]))

---
description: W&B Quickstart.
displayed_sidebar: default
---

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';

# Quickstart

Install W&B and start tracking your machine learning experiments in minutes.

## 1. Create an account and install W&B
Before you get started, make sure you create an account and install W&B:

1. [Sign up](https://wandb.ai/site) for a free account at [https://wandb.ai/site](https://wandb.ai/site) and then login to your wandb account.  
2. Install the wandb library on your machine in a Python 3 environment using [`pip`](https://pypi.org/project/wandb/).  


The following code snippets demonstrate how to install and log into W&B using the W&B CLI and Python Library:

<Tabs
  defaultValue="notebook"
  values={[
    {label: 'Notebook', value: 'notebook'},
    {label: 'Command Line', value: 'cli'},
  ]}>
  <TabItem value="cli">

Install the CLI and Python library for interacting with the Weights and Biases API:

```
pip install wandb
```

  </T

In [20]:
print(spanish_md)



---
description: Inicio rápido de W&B.
displayed_sidebar: default
---

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';

# Inicio rápido

Instala W&B y comienza a realizar el seguimiento de tus experimentos de aprendizaje automático en minutos.

## 1. Crea una cuenta e instala W&B
Antes de empezar, asegúrate de crear una cuenta e instalar W&B:

1. [Regístrate](https://wandb.ai/site) para obtener una cuenta gratuita en [https://wandb.ai/site](https://wandb.ai/site) y luego inicia sesión en tu cuenta de wandb.
2. Instala la biblioteca wandb en tu máquina en un entorno Python 3 utilizando [`pip`](https://pypi.org/project/wandb/).


Los siguientes fragmentos de código demuestran cómo instalar e iniciar sesión en W&B utilizando la CLI de W&B y la Biblioteca Python:

<Tabs
  defaultValue="notebook"
  values={[
    {label: 'Notebook', value: 'notebook'},
    {label: 'Línea de Comandos', value: 'cli'},
  ]}>
  <TabItem value="cli">

Instala la CLI y la biblioteca Python 